<a href="https://colab.research.google.com/github/MarkStephens060482/Big-Data-projects/blob/main/Assignment_1_%20Data%20Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Severity of Road Accidents
### Mark Stephens

### Collating and cleaning data sets


In [ ]:
!pip install httpx asyncio pyspark pyproj

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=45ace9631ab77034d919cf72031a2a411db91984683179fc5d4ea02f208b6812
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
import pandas as pd
import numpy as np
import csv
import httpx
import asyncio
import sys
import json
import datetime as dt
import os, glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#_________The data files are to be loaded into sub folder of the current working directory called 'data'________
PATH = '/content/drive/MyDrive/Big Data and Industry Project/'
# load the data into a pandas dataframe
files = glob.glob(PATH + "*.csv")
dfs=[]
# read the csv files one at a time and concatenate to a dataframe.
for file in files:
    with open(file,encoding="utf-8") as dataFile:
        df = pd.read_csv(dataFile,index_col=False)
        dfs.append(df)
crash_df = dfs[1]
fatal_df = dfs[0]

***
### Inspect and clean Road Crashes for Five Years Victoria data set

Force all empty strings to return **NaN** values

In [ ]:
crash_df.replace(r'^\s*$', np.nan, regex=True, inplace = True)
fatal_df.replace(r'^\s*$', np.nan, regex=True, inplace = True)

In [ ]:
crash_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60689 entries, 0 to 60688
Data columns (total 65 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   X                  60689 non-null  float64
 1   Y                  60689 non-null  float64
 2   OBJECTID           60689 non-null  int64  
 3   ACCIDENT_NO        60689 non-null  object 
 4   ABS_CODE           60689 non-null  object 
 5   ACCIDENT_STATUS    60689 non-null  object 
 6   ACCIDENT_DATE      60689 non-null  object 
 7   ACCIDENT_TIME      60689 non-null  object 
 8   ALCOHOLTIME        60689 non-null  object 
 9   ACCIDENT_TYPE      60689 non-null  object 
 10  DAY_OF_WEEK        58987 non-null  object 
 11  DCA_CODE           60689 non-null  object 
 12  HIT_RUN_FLAG       60689 non-null  object 
 13  LIGHT_CONDITION    60689 non-null  object 
 14  POLICE_ATTEND      60689 non-null  object 
 15  ROAD_GEOMETRY      60689 non-null  object 
 16  SEVERITY           606

A number of missing values are observed. The proportion of missing values per column is given below:

In [ ]:
# produce a dataframe of proportion of missing values per column.
missing_values_df = pd.DataFrame(round(100*crash_df.isnull().sum()/len(crash_df),3),
             columns = ['missing values (%)'])
#Print the columns with missing values
missing_values_df[missing_values_df.values>0]

,missing values (%)
DAY_OF_WEEK,2.804
SEVERITY,0.005
NODE_TYPE,0.147
LGA_NAME,0.982
REGION_NAME,0.989
NO_OF_VEHICLES,0.007
HEAVYVEHICLE,0.007
PASSENGERVEHICLE,0.007
MOTORCYCLE,0.007
PUBLICVEHICLE,0.007


In [ ]:
crash_df.describe()

,X,Y,OBJECTID,NODE_ID,LONGITUDE,LATITUDE,VICGRID_X,VICGRID_Y,TOTAL_PERSONS,INJ_OR_FATAL,...,PED_CYCLIST_13_18,OLD_PEDESTRIAN,OLD_DRIVER,YOUNG_DRIVER,UNLICENCSED,NO_OF_VEHICLES,HEAVYVEHICLE,PASSENGERVEHICLE,MOTORCYCLE,PUBLICVEHICLE
count,6.068900e+04,6.068900e+04,60689.00000,60689.000000,60689.000000,60689.000000,6.068900e+04,6.068900e+04,60689.000000,60689.000000,...,60689.000000,60689.000000,60689.000000,60689.000000,60689.000000,60685.000000,60685.000000,60685.000000,60685.000000,60685.000000
mean,2.497358e+06,2.421922e+06,30345.00000,234411.896934,144.732945,-37.645751,2.497358e+06,2.421922e+06,2.351629,1.308211,...,0.015687,0.024139,0.070771,0.288949,0.034471,1.790854,0.046321,1.411815,0.155689,0.012441
std,7.430158e+04,6.436034e+04,17519.54958,140607.093121,5.949869,1.534793,7.430158e+04,6.436034e+04,1.511034,0.782798,...,0.130344,0.157615,0.262288,0.510743,0.183966,0.768843,0.222298,0.931609,0.372030,0.112028
min,2.128750e+06,2.274524e+06,1.00000,4.000000,-1.000000,-39.023993,2.128750e+06,2.274524e+06,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2.485223e+06,2.393623e+06,15173.00000,51684.000000,144.830339,-37.957173,2.485223e+06,2.393623e+06,2.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000
50%,2.501454e+06,2.409539e+06,30345.00000,303841.000000,145.015981,-37.814612,2.501454e+06,2.409539e+06,2.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1.000000,0.000000,0.000000
75%,2.518897e+06,2.423416e+06,45517.00000,326827.000000,145.214600,-37.689004,2.518897e+06,2.423416e+06,3.000000,1.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000,2.000000,0.000000,0.000000
max,2.920148e+06,2.820900e+06,60689.00000,756136.000000,149.757513,-1.000000,2.920148e+06,2.820900e+06,89.000000,41.000000,...,5.000000,3.000000,3.000000,5.000000,2.000000,21.000000,4.000000,19.000000,5.000000,2.000000


It is noticed that **LATITUDE** and **LONGITUDE** variables have an instance with coordinate of -1. These are obviously errorneous instances. The corresponding projected coordinates for these instances are acquired. The **pyProj** package is used to determine the corresponding Latitude and Longitude coordinates to impute the errorneous values.

In [ ]:
proj_coord = crash_df[(crash_df['LATITUDE'] == -1)|(crash_df['LONGITUDE'] == -1)][['X','Y','LONGITUDE','LATITUDE']]
proj_coord

,X,Y,LONGITUDE,LATITUDE
78,2.516580e+06,2.396051e+06,-1.0,-1.000000
860,2.500047e+06,2.415771e+06,-1.0,-37.759056
1341,2.496381e+06,2.410668e+06,-1.0,-1.000000
2763,2.592490e+06,2.388162e+06,-1.0,-1.000000
3200,2.278978e+06,2.361679e+06,-1.0,-1.000000
...,...,...,...,...
55402,2.530035e+06,2.384499e+06,-1.0,-1.000000
57223,2.497927e+06,2.403226e+06,-1.0,-1.000000
57281,2.536155e+06,2.568068e+06,-1.0,-1.000000
58212,2.497756e+06,2.411854e+06,-1.0,-1.000000


The identified instances with erroneous values are imputed with the correct longitude and latitude values.

In [ ]:
import pyproj
def gda94_to_lon_lat(easting, northing):
    # Define the GDA94 projection using the EPSG code, Victoria Australia
    gda94 = pyproj.CRS.from_epsg(3111)
    # Define the WGS84 projection for the output (longitude and latitude)
    wgs84 = pyproj.CRS.from_epsg(4326)

    # Create transformer objects using the CRS
    transformer = pyproj.Transformer.from_crs(gda94, wgs84)

    lat, lon = transformer.transform(easting, northing)

    return pd.Series({'LONGITUDE': lon, 'LATITUDE': lat})
# Get the correct values
proj_coord[['LONGITUDE','LATITUDE']] = proj_coord.apply(lambda row: gda94_to_lon_lat(row['X'],row['Y']),axis = 1)
# impute missing values
crash_df.loc[proj_coord.index,'LONGITUDE']  = proj_coord['LONGITUDE'].values
crash_df.loc[proj_coord.index,'LATITUDE']  = proj_coord['LATITUDE'].values
#check if values are imputed correctly
crash_df.describe()

,X,Y,OBJECTID,NODE_ID,LONGITUDE,LATITUDE,VICGRID_X,VICGRID_Y,TOTAL_PERSONS,INJ_OR_FATAL,...,PED_CYCLIST_13_18,OLD_PEDESTRIAN,OLD_DRIVER,YOUNG_DRIVER,UNLICENCSED,NO_OF_VEHICLES,HEAVYVEHICLE,PASSENGERVEHICLE,MOTORCYCLE,PUBLICVEHICLE
count,6.068900e+04,6.068900e+04,60689.00000,60689.000000,60689.000000,60689.000000,6.068900e+04,6.068900e+04,60689.000000,60689.000000,...,60689.000000,60689.000000,60689.000000,60689.000000,60689.000000,60685.000000,60685.000000,60685.000000,60685.000000,60685.000000
mean,2.497358e+06,2.421922e+06,30345.00000,234411.896934,144.971124,-37.700705,2.497358e+06,2.421922e+06,2.351629,1.308211,...,0.015687,0.024139,0.070771,0.288949,0.034471,1.790854,0.046321,1.411815,0.155689,0.012441
std,7.430158e+04,6.436034e+04,17519.54958,140607.093121,0.836535,0.582661,7.430158e+04,6.436034e+04,1.511034,0.782798,...,0.130344,0.157615,0.262288,0.510743,0.183966,0.768843,0.222298,0.931609,0.372030,0.112028
min,2.128750e+06,2.274524e+06,1.00000,4.000000,140.966488,-39.023993,2.128750e+06,2.274524e+06,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2.485223e+06,2.393623e+06,15173.00000,51684.000000,144.832217,-37.957377,2.485223e+06,2.393623e+06,2.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000
50%,2.501454e+06,2.409539e+06,30345.00000,303841.000000,145.016477,-37.814770,2.501454e+06,2.409539e+06,2.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1.000000,0.000000,0.000000
75%,2.518897e+06,2.423416e+06,45517.00000,326827.000000,145.214913,-37.689541,2.518897e+06,2.423416e+06,3.000000,1.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000,2.000000,0.000000,0.000000
max,2.920148e+06,2.820900e+06,60689.00000,756136.000000,149.757513,-34.038085,2.920148e+06,2.820900e+06,89.000000,41.000000,...,5.000000,3.000000,3.000000,5.000000,2.000000,21.000000,4.000000,19.000000,5.000000,2.000000


The values for **LONGITUDE** and **LATITUDE** now have the appropriate summary statistics.

The variable **DAY_OF_WEEK** has the largest proportion of missing values of 2.9%. These missing values can be imputed by extracting days of the week from the date given in **ACCIDENT_DATE** column. Also, the variable **SEVERITY** has a minor occurrence of missing values of 0.005%. These missing values can be imputed from the counts in the variables **FATALITY**, **SERIOUSINJURY**, **OTHERINJURY** and **NONINJURED**.

Clean datetime columns seperating them into a date column and time column respectively. Also, change the format of the date from $yyyy/mm/dd$ to $yyyy-mm-dd$.

In [ ]:
# Cleaning date and time columns
crash_df['ACCIDENT_DATE'] = pd.to_datetime(crash_df['ACCIDENT_DATE'],format = '%Y-%m-%d').dt.date

crash_df['ACCIDENT_TIME'] = pd.to_datetime(crash_df['ACCIDENT_TIME']).dt.time

Fill in missing values of **DAY_OF_WEEK** using **ACCIDENT_DATE**, apply to whole data set.

In [ ]:
# give the day of the week for each date of accident
crash_df['DAY_OF_WEEK'] = crash_df['ACCIDENT_DATE'].apply(lambda x: x.strftime('%A'))

Define the rules in order to form the correct classification label for the missing instances of **SEVERITY**

In [ ]:
# define function to impute severity of accident.
def impute_severity(row):
  if row['FATALITY'] >= 1:
    return 'Fatal accident'
  elif row['SERIOUSINJURY'] >= 1:
    return 'Serious injury accident'
  elif row['OTHERINJURY'] >= 1:
    return 'Other injury accident'
  else:
    return 'Non injury accident'

# filter out instances with missing values.
severity_class = crash_df[crash_df.SEVERITY.isnull()][['SEVERITY','FATALITY','SERIOUSINJURY','OTHERINJURY']]
severity_class

,SEVERITY,FATALITY,SERIOUSINJURY,OTHERINJURY
9182,NaN,0,0,1
11941,NaN,0,1,0
59208,NaN,0,0,1


impute missing values

In [ ]:
severity_class['SEVERITY'] = severity_class.apply(impute_severity, axis = 1)
crash_df.loc[severity_class.index,'SEVERITY']  = severity_class['SEVERITY'].values
#inspect values
crash_df.loc[severity_class.index,'SEVERITY']

9182       Other injury accident
11941    Serious injury accident
59208      Other injury accident
Name: SEVERITY, dtype: object

The categorical variables **LGA_NAME** and **REGION_NAME** have approximately 1% missing values, while their counterpart variable **LGA_NAME_ALL** and **REGION_NAME_ALL** respectively have either no or very few missing values. The variable values are to be compared and non-missing values for **LGA_NAME_ALL** and **REGION_NAME_ALL** are to be used to imputed missing values for **LGA_NAME** and **REGION_NAME** respectively.   

In [ ]:
#determine indices of missing values for LGA_NAME
lga_name_missing_value_boolean_mask = crash_df['LGA_NAME'].isnull()

lga_name_impute_values = crash_df.loc[lga_name_missing_value_boolean_mask,'LGA_NAME_ALL']
# seperate the words by comma and return first value in list
lga_name_impute_values = lga_name_impute_values.map(lambda x: x.split(',')[0])
#impute missing values for LGA_NAME
crash_df.loc[lga_name_missing_value_boolean_mask,'LGA_NAME']= lga_name_impute_values

#determine indices of missing values for REGION_NAME
region_name_missing_value_boolean_mask = crash_df['REGION_NAME'].isnull()
region_name_impute_values = crash_df.loc[region_name_missing_value_boolean_mask,'REGION_NAME_ALL']
# seperate the words by comma and return first value in list
region_name_impute_values = region_name_impute_values.map(lambda x: x.split(',')[0])
#impute missing values for LGA_NAME
crash_df.loc[region_name_missing_value_boolean_mask,'REGION_NAME'] = region_name_impute_values

### Categorical data types
Review the unique values of the categorical variables and clean the values.

In [ ]:
print(crash_df['LGA_NAME'].unique())

['GEELONG' 'WHITTLESEA' 'MARIBYRNONG' 'CASEY' 'CORANGAMITE' 'LATROBE'
 'YARRA' 'MORNINGTON PENINSULA' 'MELBOURNE' 'MOYNE' 'WHITEHORSE' 'BENDIGO'
 'BANYULE' 'DANDENONG' 'MONASH' 'YARRA RANGES' 'BRIMBANK' 'HORSHAM'
 'GLENELG' 'EAST GIPPSLAND' 'BOROONDARA' 'MANNINGHAM' 'PORT PHILLIP'
 'HUME' 'FRANKSTON' 'WYNDHAM' 'ARARAT' 'BALLARAT' 'DAREBIN' 'MOORABOOL'
 'STONNINGTON' 'GLEN EIRA' 'NILLUMBIK' 'COLAC OTWAY' 'MOIRA' 'KINGSTON'
 'BAYSIDE' 'MOONEE VALLEY' 'MACEDON RANGES' 'INDIGO' 'MILDURA'
 'HOBSONS BAY' 'WODONGA' 'MELTON' 'MURRINDINDI' 'MORELAND' 'MITCHELL'
 'KNOX' 'SOUTH GIPPSLAND' 'BAW BAW' 'SURF COAST' 'MANSFIELD' 'PYRENEES'
 'CARDINIA' 'MAROONDAH' 'HEPBURN' 'WELLINGTON' 'CAMPASPE' 'YARRIAMBIACK'
 'BASS COAST' 'NORTHERN GRAMPIANS' '(MOUNT BULLER)' 'BENALLA' 'WANGARATTA'
 'TOWONG' 'WARRNAMBOOL' 'GOLDEN PLAINS' 'SHEPPARTON' 'MOUNT ALEXANDER'
 'ALPINE' 'SWAN HILL' 'SOUTHERN GRAMPIANS' 'GANNAWARRA' 'LODDON'
 'HINDMARSH' 'STRATHBOGIE' 'CENTRAL GOLDFIELDS' 'WEST WIMMERA'
 '(LAKE MOUNTAIN)' 'BU

Remove the special characters from the values using regular expressions.

In [ ]:
import re
def clean_string(word):
  regex =re.compile(r'\((.+)\)')
  result = regex.findall(word)
  if len(result) >0:
    return result[0]
  return word
word = '(MOUNT BAW BAW)'

crash_df['LGA_NAME'] = crash_df['LGA_NAME'].apply(clean_string)

Remove columns comprising of text that are of a non-categorical data type or have too many missing values.

In [ ]:
#drop columns of superfluous data, of text
drop_column_indx = [0,1,2,4,5,8,9,11,14,15,18,22,25,26]+[x for x in range(54,64)]
crash_df.drop(crash_df.iloc[:,drop_column_indx],axis=1, inplace = True)

Review data set information

In [ ]:
crash_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60689 entries, 0 to 60688
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ACCIDENT_NO        60689 non-null  object 
 1   ACCIDENT_DATE      60689 non-null  object 
 2   ACCIDENT_TIME      60689 non-null  object 
 3   DAY_OF_WEEK        60689 non-null  object 
 4   HIT_RUN_FLAG       60689 non-null  object 
 5   LIGHT_CONDITION    60689 non-null  object 
 6   SEVERITY           60689 non-null  object 
 7   SPEED_ZONE         60689 non-null  object 
 8   NODE_ID            60689 non-null  int64  
 9   LONGITUDE          60689 non-null  float64
 10  LATITUDE           60689 non-null  float64
 11  LGA_NAME           60689 non-null  object 
 12  REGION_NAME        60689 non-null  object 
 13  TOTAL_PERSONS      60689 non-null  int64  
 14  INJ_OR_FATAL       60689 non-null  int64  
 15  FATALITY           60689 non-null  int64  
 16  SERIOUSINJURY      606

The Statistical Division Name variable **STAT_DIV_NAME** classifies the regions to either 'Metro' or 'Country'. The missing values for STAT_DIV_NAME have corresponding LGA_NAME values that match with given values for STAT_DIV_NAME. This is used to impute the missing valiues for  STAT_DIV_NAME.

In [ ]:
values = crash_df[crash_df['STAT_DIV_NAME'].isnull()]['LGA_NAME']
state_div_name_impute_values = {}
for value in values:
  impute_value = crash_df[crash_df['LGA_NAME'] == value]['STAT_DIV_NAME'].unique()[0]
  state_div_name_impute_values[value] = impute_value
# dictionary of LGA keys and their corresponding STAT_DIV_NAME values.
state_div_name_impute_values
# impute missing values
missing_values_boolean_mask = crash_df['STAT_DIV_NAME'].isnull()
for key,value in state_div_name_impute_values.items():
  crash_df.loc[missing_values_boolean_mask & (crash_df["LGA_NAME"] == key),'STAT_DIV_NAME'] = value
print(f"number of missing values is: {crash_df['STAT_DIV_NAME'].isnull().sum()}")

number of missing values is: 0


Remove remaining null values from dataset

In [ ]:
crash_df.dropna(inplace = True)

Variables **NO_OF_VEHICLES** to **PUBLICVEHICLE** are of float type and are to be changed to integer type

In [ ]:
for idx in range(35,40):
  crash_df[crash_df.columns[idx]] = crash_df[crash_df.columns[idx]].astype('int64',copy=True)

In [ ]:
crash_df.head()

,ACCIDENT_NO,ACCIDENT_DATE,ACCIDENT_TIME,DAY_OF_WEEK,HIT_RUN_FLAG,LIGHT_CONDITION,SEVERITY,SPEED_ZONE,NODE_ID,LONGITUDE,...,OLD_DRIVER,YOUNG_DRIVER,ALCOHOL_RELATED,UNLICENCSED,NO_OF_VEHICLES,HEAVYVEHICLE,PASSENGERVEHICLE,MOTORCYCLE,PUBLICVEHICLE,STAT_DIV_NAME
0,T20150013818,2015-07-01,05:15:00,Wednesday,No,Dark Street lights on,Other injury accident,60 km/hr,293078,144.368427,...,0,0,No,0,1,0,1,0,0,Country
1,T20150013844,2015-07-01,11:30:00,Wednesday,No,Day,Other injury accident,80 km/hr,226603,145.079994,...,0,0,No,0,1,0,0,1,0,Metro
2,T20150013854,2015-07-01,13:40:00,Wednesday,No,Day,Other injury accident,50 km/hr,38756,144.894442,...,0,1,No,0,2,0,2,0,0,Metro
3,T20150013897,2015-07-02,10:31:00,Thursday,Yes,Day,Other injury accident,Camping grounds or off road,296041,145.277772,...,0,0,No,0,2,0,2,0,0,Metro
4,T20150013907,2015-07-02,10:25:00,Thursday,No,Day,Serious injury accident,100 km/hr,293455,143.251106,...,0,0,No,0,1,0,1,0,0,Country


Set the ACCIDENT_NO as the index

In [ ]:
crash_df.set_index('ACCIDENT_NO', drop = True, inplace = True)

Write the cleaned crash data to a csv file

In [ ]:
# Write the combined DataFrame to a CSV file
crash_df.to_csv(PATH + '/project cleaned data/crash_cleaned.csv', index = False)

### Inspect and clean Fatal Crashes - Lives Lost in the Last 5 Years in Victoria data set

In [ ]:
fatal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264 entries, 0 to 1263
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   X           1264 non-null   float64
 1   Y           1264 non-null   float64
 2   OBJECTID    1264 non-null   int64  
 3   ACCIDENT_N  1264 non-null   object 
 4   ACCIDENT_D  1264 non-null   object 
 5   ACC_TIME    1264 non-null   object 
 6   NO_VEHICLE  1264 non-null   int64  
 7   ACCIDENT_T  1264 non-null   int64  
 8   ACC_T_DESC  1264 non-null   object 
 9   ABS_CODE    1264 non-null   object 
 10  PERSON_ID   1264 non-null   object 
 11  AGE         1259 non-null   object 
 12  RD_USER_T   1264 non-null   int64  
 13  RD_U_DESC   1264 non-null   object 
 14  SEX         1264 non-null   object 
 15  LOCATION_D  1264 non-null   object 
 16  NODE_ID     1264 non-null   int64  
 17  LOCATION_T  1264 non-null   object 
 18  NODE_TYPE   1264 non-null   object 
 19  N_TYPE_DES  1264 non-null  

Majority of features give data that are same or equivalent to features in *Road Crashes for 5 Years Victoria* data set. Extract features providing additional information. Also, set the variable **ACCIDENT_NO** to index.

In [ ]:
#drop columns of superfluous data, of text
fatal_clean_df = fatal_df[['ACCIDENT_N','AGE','LOCATION_D']].copy()
fatal_clean_df.rename(columns={'ACCIDENT_N': 'ACCIDENT_NO'},inplace = True)
fatal_clean_df.set_index('ACCIDENT_NO', drop = True, inplace = True)
fatal_clean_df.head()

,AGE,LOCATION_D
ACCIDENT_NO,,
T20150013823,39,At Int of DANDENONG ROAD and ATKINSON STREET
T20150013938,18,On HAMILTON HIGHWAY 902m W from Int BARUNAH PL...
T20150014033,46,At Int of CAHILLS ROAD and OLD WILBY ROAD
T20150014033,65,At Int of CAHILLS ROAD and OLD WILBY ROAD
T20150014077,49,On HUME FREEWAY 34m from BRIDGE OVER NTH EASTE...


In [ ]:
# Write the combined DataFrame to a CSV file
fatal_clean_df.to_csv(PATH + '/project cleaned data/fatal_cleaned.csv', index = True)

### Define Weather data set

In [ ]:
weather_df = crash_df[['ACCIDENT_DATE','ACCIDENT_TIME','LATITUDE','LONGITUDE']].copy()
weather_df

,ACCIDENT_DATE,ACCIDENT_TIME,LATITUDE,LONGITUDE
ACCIDENT_NO,,,,
T20150013818,2015-07-01,05:15:00,-38.177369,144.368427
T20150013844,2015-07-01,11:30:00,-37.656052,145.079994
T20150013854,2015-07-01,13:40:00,-37.816007,144.894442
T20150013897,2015-07-02,10:31:00,-38.081612,145.277772
T20150013907,2015-07-02,10:25:00,-38.015857,143.251106
...,...,...,...,...
T20200023346,2019-06-06,15:00:00,-38.493332,143.889108
T20210004913,2017-04-18,12:01:00,-37.825929,145.000038
T20210008589,2019-03-23,10:21:00,-37.815726,145.178806


Define weather api to call in weather observations

In [ ]:
import math, time
import pandas as pd
import csv
import httpx
import asyncio
import sys
import json
import datetime as dt
import logging

# Define global configuration variables
config = {
    'max_concurrent_requests': 100, # Adjust the value based on fair use limits
     #'time_period_minutes': 10,  # Adjust the value based on fair use limits
    'batch_size': 100,
    'max_retries': 3,
    'api_keys': ['TUTZUAHPWJYWYFGHTEYCX4439', 'C44AY3ZUJJBK4JWZMUHJAQWRT', 'PXX6MMTB32NJ7JDRQ3PRFYSEW'],
    'max_rate_limit':10 # requests per second
}

#define the fuction to perform api calls to get historical weather data.
async def get_weather_in_batches(batch, api_keys,max_retries):
    '''
    '''
    async with httpx.AsyncClient() as client:
      for retry in range(max_retries + 1):
        try:
          batch_results = []
          for row_count,row in enumerate(batch.to_dict(orient="records")):
            # Create the query parameters based on row features and the API key
            ApiQuery = create_query(row)

            # sequentially assign an API key based on the batch
            api_key = api_keys[row_count % len(api_keys)]
            #query parameters
            url_params = {'unitGroup':'metric', #UnitGroup sets the units of the output - SI metrics
                          'contentType':'json',
                          'include':'current', #Include current conditions of weather
                          'maxDistance':'25000',#Interpolate weather from station within max distance of 2km
                          'maxStations':'3', #Interpolate weather from 3 stations within max distance.
                          #'allowAsynch':'true', # allow synchronous requests
                          'aggregateHours':'0',
                          'aggregateMinutes':'15',
                          'combinationMethod':'aggregate',
                          'key':api_key
                          }
            # Send the API request with query parameters
            response = await client.get(ApiQuery, params = url_params)
            # Raise an exception if the response status code is an error
            response.raise_for_status()
            # Convert the response to JSON
            json_weatherdata = response.json()
            #add result to batch
            batch_results.append(json_weatherdata['currentConditions'])
            # Calculate the delay based on the maximum rate limit
            delay = 1 / config['max_rate_limit']
            # Introduce the delay before making the next request
            await asyncio.sleep(delay)

          return batch_results
        #error handling
        except httpx.HTTPError as e:
          if e.response.status_code == 429:
            print("Fair use limit exceeded. Retrying the batch...")
            #Retry the entire batch
            continue
          else:
            print(f"HTTP error occurred: {e}")
            break  # Break the retry loop for other HTTP errors
        except Exception as e:
              print(f"Exception occurred: {e}")
              break  # Break the retry loop for other errors

    # Return empty results if all retries failed
    return[]

# Create the query parameters based on row features.
def create_query(row):
  '''
  '''
  #API base url
  BaseURL = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'
  # Modify this function based on the structure of your API and DataFrame
  Location = f"{row['LATITUDE']},{row['LONGITUDE']}"
  date = row['ACCIDENT_DATE']
  time = row['ACCIDENT_TIME']
  startDateTime = f"{date}T{time}"
  #basic query including location
  ApiQuery = f"{BaseURL}{Location}/{startDateTime}"
  return ApiQuery

#Produce a function that splits the dataframe into chunks of the specified size.
def split_df_chunks(data_df, chunk_size):
    total_length = len(data_df)
    total_chunk_num = math.ceil(total_length / chunk_size)
    chunks = []

    for i in range(0, total_length, chunk_size):
        chunk = data_df[i : i + chunk_size]
        chunks.append(chunk)

    return chunks


# Function to process the DataFrame rows asynchronously
async def process_batches(data_df, api_keys, max_retries, batch_size):
  """
  """
  total_length = len(data_df)
  semaphore = asyncio.Semaphore(config['max_concurrent_requests'])
  tasks = []

  # split df into batches
  batches = split_df_chunks(data_df,batch_size)

  request_number = 0
  elapsed_time = 0
  for batch in batches:
    # Start time of batch processing
    start_time = time.perf_counter()
    # Perform chunk API call in loop with retry batch method
    async with semaphore:
          # perform batch API tasks in loop
          results = asyncio.create_task(get_weather_in_batches(batch, api_keys,max_retries))
          tasks.append(results)

    # Wait for all tasks to complete
    results = await asyncio.gather(*tasks)
    # End time of batch processing
    end_time = time.perf_counter()
    elapsed_time += (end_time - start_time)
    request_number += len(batch)
    # Print the progress of successfully processed batches
    print(f"Processed {request_number*100/total_length: 0.2f}% of total requests successfully.")
    print(f"Total time taken to process batches: {elapsed_time: 0.4f} seconds")

  return results


### Main Code ###
async def main(df):
  start_time = time.perf_counter()
  # Perform asynchronous processing on each partition in batches
  results = await process_batches(df,
                                  api_keys = config['api_keys'],
                                  max_retries = config['max_retries'],
                                  batch_size = config['batch_size'])

  # Convert the results to Row objects with the defined schema
  Flatten_results = [result for batch_results in results for result in batch_results]

  # Create DataFrame using the rows with schema
  df_with_results = pd.DataFrame(Flatten_results, index = df.index)

  # Append the new DataFrame to the original DataFrame
  output_df = pd.concat([df, df_with_results], axis=1,copy = True)
  end_time =  time.perf_counter()
  time_duration = end_time - start_time
  print(f'Total computation time is : {time_duration: 0.1f} seconds')
  print(f'Average computation time is : {time_duration/len(df_with_results): 0.1f} seconds per row')
  return output_df

if __name__ == '__main__':

  output_df1 = await main(weather_df)

In [ ]:
# Write the combined DataFrame to a CSV file
output_df1.to_csv(PATH + '/project cleaned data/weather.csv', index = False)

The weather data is to be cleaned, checking and casting the correct data types, filling in or removing null values, dropping irrelevant or unnecessary variable columns.

In [ ]:
with open(PATH + '/project cleaned data/weather.csv', encoding="utf-8") as dataFile:
  weather_df = pd.read_csv(dataFile,
                           parse_dates={'ACCIDENT_DATETIME':['ACCIDENT_DATE','ACCIDENT_TIME']},
                           index_col = ['ACCIDENT_NO'])
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60683 entries, T20150013818 to T20220019338
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ACCIDENT_DATETIME  60683 non-null  datetime64[ns]
 1   LATITUDE           60683 non-null  float64       
 2   LONGITUDE          60683 non-null  float64       
 3   datetime           60683 non-null  object        
 4   datetimeEpoch      60683 non-null  int64         
 5   temp               60683 non-null  float64       
 6   feelslike          60683 non-null  float64       
 7   humidity           60636 non-null  float64       
 8   dew                60636 non-null  float64       
 9   precip             60683 non-null  float64       
 10  precipprob         60683 non-null  int64         
 11  snow               48110 non-null  float64       
 12  snowdepth          48110 non-null  float64       
 13  preciptype         1331 non-null   object       

In [ ]:
output_df1['ACCIDENT_DATETIME'] = pd.to_datetime(output_df1['ACCIDENT_DATE'].apply(str)+' '+output_df1['ACCIDENT_TIME'].apply(str))
output_df2 = output_df1.drop(columns = ['ACCIDENT_DATE','ACCIDENT_TIME']).copy()
weather_df2 = pd.concat([output_df2,weather_df],axis = 0)

In [ ]:
weather_df = weather_df2.copy()
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60685 entries, T20170020514 to T20220019338
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   LATITUDE           60685 non-null  float64       
 1   LONGITUDE          60685 non-null  float64       
 2   datetime           60685 non-null  object        
 3   datetimeEpoch      60685 non-null  int64         
 4   temp               60685 non-null  float64       
 5   feelslike          60685 non-null  float64       
 6   humidity           60638 non-null  float64       
 7   dew                60638 non-null  float64       
 8   precip             60685 non-null  float64       
 9   precipprob         60685 non-null  float64       
 10  snow               48112 non-null  float64       
 11  snowdepth          48112 non-null  float64       
 12  preciptype         1331 non-null   object        
 13  windgust           121 non-null    float64      

Drop columns compring of irrelevant text or superfluous information.

In [ ]:
#drop columns of superfluous data, of text
drop_column_indx = [2,3,9,10,11,12,13,16,17,19,20,21,22,23,24,25,26,27,28,29,32]
weather_df.drop(weather_df.iloc[:,drop_column_indx],axis=1, inplace = True)

A number of missing observations have occurred. The proportion of missing values is given below:

In [ ]:
# produce a dataframe of proportion of missing values per column.
pd.DataFrame(round(100*weather_df.isnull().sum()/len(weather_df),3),
             columns = ['missing values (%)'])

,missing values (%)
LATITUDE,0.000
LONGITUDE,0.000
temp,0.000
feelslike,0.000
humidity,0.077
dew,0.077
precip,0.000
windspeed,0.000
winddir,0.000
cloudcover,0.000


The weather condition variables of numerical continuous data type are summarised as following:

In [ ]:
weather_df.describe()

,LATITUDE,LONGITUDE,temp,feelslike,humidity,dew,precip,windspeed,winddir,cloudcover,moonphase
count,60685.000000,60685.000000,60685.000000,60685.000000,60638.000000,60638.000000,60685.000000,60685.000000,60685.000000,60685.000000,60685.000000
mean,-37.700734,144.971150,16.465807,16.145827,65.183947,8.915500,0.061898,14.377487,204.341567,49.160901,0.483977
std,0.582625,0.836548,6.520916,6.721263,20.178597,4.110535,0.691313,8.604810,107.339428,39.454439,0.288182
min,-39.023993,140.966488,-5.100000,-11.100000,4.900000,-22.400000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-37.957391,144.832271,11.900000,11.900000,51.700000,6.000000,0.000000,8.100000,140.000000,5.400000,0.250000
50%,-37.814770,145.016502,15.600000,15.600000,66.300000,8.600000,0.000000,13.000000,210.000000,50.000000,0.500000
75%,-37.689543,145.214946,20.300000,20.300000,80.750000,11.600000,0.000000,19.300000,290.000000,91.300000,0.750000
max,-34.038085,149.757513,46.300000,46.300000,100.000000,23.700000,60.528000,85.200000,360.000000,100.000000,0.980000


The precipitation variable is heavily skewed. Humidity and dew have missing values. Missing values in the dew and humidity columns are to be imputed using Multiple Imputations by Chained Equations (MICE) algorithm.

In [ ]:
# need to enable iterative imputer explicitly since its still experimental
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(random_state=100, max_iter=20)

#select numerical variables only
df_train = weather_df.select_dtypes(include=np.number)

# fit on the dataset
imputer.fit(df_train)

#impute missing values
df_imputed = imputer.transform(df_train)

# Assign the imputed values back to the original data set
weather_df.loc[:,df_train.columns] = df_imputed

weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60685 entries, T20170020514 to T20220019338
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   LATITUDE           60685 non-null  float64       
 1   LONGITUDE          60685 non-null  float64       
 2   temp               60685 non-null  float64       
 3   feelslike          60685 non-null  float64       
 4   humidity           60685 non-null  float64       
 5   dew                60685 non-null  float64       
 6   precip             60685 non-null  float64       
 7   windspeed          60685 non-null  float64       
 8   winddir            60685 non-null  float64       
 9   cloudcover         60685 non-null  float64       
 10  moonphase          60685 non-null  float64       
 11  ACCIDENT_DATETIME  60685 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(11)
memory usage: 8.0+ MB


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


### Public Holiday and Long Weekends data
Dates that are identified as Public Holidays or Long Weekends for Victoria are identified using API call. A *Yes* or *No* for a variable **PUBLICHOLIDAY_LONGWEEKEND** is produced for each record based on **ACCIDENT_DATE** value.

In [ ]:
import requests
import numpy as np

# Produce array of years from data
data_years = list(np.unique(crash_df['ACCIDENT_DATE'].apply(lambda x: x.year)))
holiday_type = ['LongWeekend','PublicHolidays']

# Function to perform API call for a single record
def perform_api_calls(year,holiday_type):
  #API endpoint
  base_url1 = 'https://date.nager.at/api/v3/LongWeekend'
  base_url2 = 'https://date.nager.at/api/v3/PublicHolidays'

  # Build the API request URL based on the year
  url1 = f'{base_url1}/{year}/AU?'
  url2 = f'{base_url2}/{year}/AU?'

  data = {}
  for i,url in enumerate([url1,url2]):
    # Make the API call
    response = requests.get(url)
    # Process the API response
    if response.status_code == 200:
      data[holiday_type[i]] = response.json()
    # Process the data as needed
    else:
      print("The request is denied.")
      #Retry the call
      continue
  #return a dictionary of json outputs
  return data

#function to process the data acquired from the API and extract the relevant features
def process_holidays(data_dict,holiday_type):
  dfs = []
  for key,values in data_dict.items():
    if key == holiday_type[0]:
      output = pd.DataFrame(values)
      output.drop(columns = ['dayCount','needBridgeDay'], inplace = True)
    else:
      list_processed_data = []
      for value in values:
        processed_data = dict()
        if value["counties"] == None:
          processed_data['startDate'] = value['date']
          processed_data['endDate'] = value['date']
          list_processed_data.append(processed_data)
        elif "AU-VIC" in value['counties']:
          processed_data['startDate'] = value['date']
          processed_data['endDate'] = value['date']
          list_processed_data.append(processed_data)
      output = pd.DataFrame(list_processed_data)
    dfs.append(output)
  return pd.concat(dfs)

###main code###
if __name__ == '__main__':
  #loop through each unique year of the crash data set
  year_df = []
  for year in data_years:
    # Perform API calls to request holiday data for each year
    data_dict = perform_api_calls(year,holiday_type)
    # Produce a dataframe of holiday periods
    df = process_holidays(data_dict,holiday_type)
    year_df.append(df)
  holiday_df = pd.concat(year_df)
  holiday_df.drop_duplicates(subset=['startDate'],inplace = True)
  holiday_df.drop_duplicates(subset=['endDate'],inplace = True)
  holiday_df.reset_index(inplace = True, drop = True)



Create a variable **PUBLICHOLIDAY_LONGWEEKEND** in the crash dataframe based on the condition if the incident occurs on or during **startDate** and **endDate** of the holiday dataframe, then return a "Yes" value else return a "No" value.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
# Create SparkSession
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('HolidayApi') \
    .getOrCreate()

sc = spark.sparkContext

# Convert datetimes to integers
holiday_df['startDate'] = pd.to_datetime(holiday_df['startDate'])
holiday_df['endDate'] = pd.to_datetime(holiday_df['endDate'])
#convert dataframe to dictionary
holiday_df2 = holiday_df.to_dict('records')
# convert crash ACCIDENT_DATE to integer and store as RDD.
crash_dates_df = crash_df.reset_index()[['ACCIDENT_NO','ACCIDENT_DATE']].copy()
crash_dates_df['ACCIDENT_DATE'] = pd.to_datetime(crash_dates_df['ACCIDENT_DATE'])
# Convert to a pyspark dataframe then to rdd
crash_dates_rdd = spark.createDataFrame(crash_dates_df).rdd

#function to determine if the dat is a public holiday or longweekend
def determine_holiday(x,holiday_dict):
  result = np.sum([1 if (row['startDate'] <= x <= row['endDate']) else 0 for row in holiday_dict])
  if result == 1:
    return 'Yes'
  return 'No'

#apply function to crash dataset and add as a new column
crash_dates_yesno_rdd = crash_dates_rdd.map(
    lambda x: (x.ACCIDENT_NO,x.ACCIDENT_DATE ,determine_holiday(x.ACCIDENT_DATE ,holiday_df2))).toDF(['ACCIDENT_NO','ACCIDENT_DATE','PUBLICHOLIDAY_LONGWEEKEND'])
# convert back to pandas dataframe
crash_dates_yesno_df = crash_dates_yesno_rdd.toPandas()
crash_dates_yesno_df['ACCIDENT_DATE'] = crash_dates_yesno_df['ACCIDENT_DATE'].astype('datetime64[ns]')
crash_dates_yesno_df.tail()
# Cleaning date and time columns

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,ACCIDENT_NO,ACCIDENT_DATE,PUBLICHOLIDAY_LONGWEEKEND
60680,T20200023346,2019-06-06,No
60681,T20210004913,2017-04-18,No
60682,T20210008589,2019-03-23,No
60683,T20220007525,2018-10-27,No
60684,T20220019338,2019-03-01,No


In [ ]:
#set ACCIDENT_NO as index
crash_dates_yesno_df.set_index('ACCIDENT_NO', drop = True, inplace = True)
# Write the combined DataFrame to a CSV file
crash_dates_yesno_df.to_csv(PATH + '/project cleaned data/holiday_yesno.csv', index = False)

### Combining Cleaned Datasets
The individual cleaned datasets are collected and merged along indices.

In [ ]:
#remove the common columns from the other dataframes
crash_dates_yesno_df.drop(columns = 'ACCIDENT_DATE',inplace = True)
weather_df.drop(columns = ['ACCIDENT_DATETIME', 'LATITUDE', 'LONGITUDE'], inplace = True)

complete_df = pd.concat([crash_df,crash_dates_yesno_df,weather_df], axis = 1)
complete_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60685 entries, T20150013818 to T20220019338
Data columns (total 50 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ACCIDENT_DATE              60685 non-null  object 
 1   ACCIDENT_TIME              60685 non-null  object 
 2   DAY_OF_WEEK                60685 non-null  object 
 3   HIT_RUN_FLAG               60685 non-null  object 
 4   LIGHT_CONDITION            60685 non-null  object 
 5   SEVERITY                   60685 non-null  object 
 6   SPEED_ZONE                 60685 non-null  object 
 7   NODE_ID                    60685 non-null  int64  
 8   LONGITUDE                  60685 non-null  float64
 9   LATITUDE                   60685 non-null  float64
 10  LGA_NAME                   60685 non-null  object 
 11  REGION_NAME                60685 non-null  object 
 12  TOTAL_PERSONS              60685 non-null  int64  
 13  INJ_OR_FATAL               60685 

Have the potential target variable **SEVERITY** moved to the end of the dataframe.

In [ ]:
column_to_move = complete_df.pop("SEVERITY")

# insert column with insert(location, column_name, column_value)

complete_df.insert(len(complete_df.columns), "SEVERITY", column_to_move)

Perform a value count on the classes of the **SEVERITY** variable.

In [ ]:
complete_df['SEVERITY'].value_counts()

Other injury accident      35359
Serious injury accident    24052
Fatal accident              1274
Name: SEVERITY, dtype: int64

In this case, it is practical to handle the severe class imbalance, with the fourth class having an extremely low proportion of instances, by merging the **Non injury accident** class with **Other injury accident** class.

In [ ]:
complete_df.loc[complete_df['SEVERITY'] == 'Non injury accident','SEVERITY'] = 'Other injury accident'

Write the complete data set to csv file

In [ ]:
# Write the combined DataFrame to a CSV file
df.to_csv(PATH + '/project cleaned data/complete project data.csv', index = True)